In [1]:
print('Lalala')

Lalala


In [2]:
print(spark.conf.get("spark.sql.extensions"))
print(spark.conf.get("spark.sql.catalog.spark_catalog"))

io.delta.sql.DeltaSparkSessionExtension
org.apache.spark.sql.delta.catalog.DeltaCatalog


In [7]:
spark.sql('create database module_4')

""


In [10]:
create_DDLtable_query = \
'''
create or replace table module_4.lesson43 (
    id int,
    run_params string,
    status string,
    status_message string
)
using delta
'''

spark.sql(create_DDLtable_query)

26/02/10 12:56:50 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


26/02/10 12:57:04 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `module_4`.`lesson43` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.


""


In [15]:
spark.sql('describe formatted module_4.lesson43').show(100, False)

+----------------------------+------------------------------------------------------+-------+
|col_name                    |data_type                                             |comment|
+----------------------------+------------------------------------------------------+-------+
|id                          |int                                                   |       |
|run_params                  |string                                                |       |
|status                      |string                                                |       |
|status_message              |string                                                |       |
|                            |                                                      |       |
|# Partitioning              |                                                      |       |
|Not partitioned             |                                                      |       |
|                            |                              

In [16]:
spark.sql('describe detail module_4.lesson43').show(100, False)

+------+------------------------------------+-----------------+-----------+------------------------------------------------------+-----------------------+-------------------+----------------+--------+-----------+----------+----------------+----------------+------------------------+
|format|id                                  |name             |description|location                                              |createdAt              |lastModified       |partitionColumns|numFiles|sizeInBytes|properties|minReaderVersion|minWriterVersion|tableFeatures           |
+------+------------------------------------+-----------------+-----------+------------------------------------------------------+-----------------------+-------------------+----------------+--------+-----------+----------+----------------+----------------+------------------------+
|delta |e334da84-2ba2-401e-aa01-6a5e9610d614|module_4.lesson43|null       |s3a://yc-dataproc-data1/warehouse/module_4.db/lesson43|2026-02-10 12:56:39.6

In [3]:
spark.sql('describe history module_4.lesson43').show(100, False)

+-------+-------------------+------+--------+-----------------------+-----------------------------------------------------------------------------+----+--------+---------+-----------+--------------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-----------------------------------+
|version|timestamp          |userId|userName|operation              |operationParameters                                                          |job |notebook|clusterId|readVersion|isolationLevel|isBlindAppend|operationMetrics                                                                                                                                                                                                                 |userMetadata|engineInfo                         |
+-------+---------------

In [22]:
spark.sql('insert into module_4.lesson43 values (2, \'lala1\', \'lala2\', \'lala3\'), (3, \'lala1\', \'lala2\', \'lala3\')')

""


In [26]:
spark.table('module_4.lesson43').show()

+---+----------+------+--------------+
| id|run_params|status|status_message|
+---+----------+------+--------------+
|  2|   lalala2| lala2|         lala3|
|  1|     lala1| lala2|         lala3|
|  3|     lala1| lala2|         lala3|
+---+----------+------+--------------+



In [25]:
spark.sql('update module_4.lesson43 set run_params = \'lalala2\' where id = 2')

num_affected_rows
1


In [7]:
spark.sql('''
create or replace temp view lesson43_update as
select * from values 
  (4, 'wrong params', 'Failed', 'Error: OutOfMemoryException'),
  (1, 'upload test table', 'Success', 'Table was uploaded'),
  (3, 'correct settings', 'Success', 'Empty message'),
  (5, 'correct settings', 'Failed', 'Unavailable source cluster')
as tab(id, run_params, status, status_message)
''')

""


In [9]:
spark.sql('select * from module_4.lesson43').show()

spark.sql('select * from lesson43_update').show()

+---+----------+------+--------------+
| id|run_params|status|status_message|
+---+----------+------+--------------+
|  2|   lalala2| lala2|         lala3|
|  1|     lala1| lala2|         lala3|
|  3|     lala1| lala2|         lala3|
+---+----------+------+--------------+

+---+-----------------+-------+--------------------+
| id|       run_params| status|      status_message|
+---+-----------------+-------+--------------------+
|  4|     wrong params| Failed|Error: OutOfMemor...|
|  1|upload test table|Success|  Table was uploaded|
|  3| correct settings|Success|       Empty message|
|  5| correct settings| Failed|Unavailable sourc...|
+---+-----------------+-------+--------------------+



In [10]:
spark.sql('''
merge into module_4.lesson43
using lesson43_update
on module_4.lesson43.id = lesson43_update.id
when matched then update set *
when not matched then insert *
''')

26/02/11 04:52:10 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
26/02/11 04:52:11 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.
26/02/11 04:52:11 WARN HintErrorLogger: Hint (strategy=broadcast) is not supported in the query: build left for full outer join.


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
4,2,0,2


In [11]:
spark.sql('select * from module_4.lesson43').show()

spark.sql('select * from lesson43_update').show()

+---+-----------------+-------+--------------------+
| id|       run_params| status|      status_message|
+---+-----------------+-------+--------------------+
|  1|upload test table|Success|  Table was uploaded|
|  3| correct settings|Success|       Empty message|
|  4|     wrong params| Failed|Error: OutOfMemor...|
|  5| correct settings| Failed|Unavailable sourc...|
|  2|          lalala2|  lala2|               lala3|
+---+-----------------+-------+--------------------+

+---+-----------------+-------+--------------------+
| id|       run_params| status|      status_message|
+---+-----------------+-------+--------------------+
|  4|     wrong params| Failed|Error: OutOfMemor...|
|  1|upload test table|Success|  Table was uploaded|
|  3| correct settings|Success|       Empty message|
|  5| correct settings| Failed|Unavailable sourc...|
+---+-----------------+-------+--------------------+



In [14]:
spark.sql('describe history module_4.lesson43')

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
4,2026-02-11 04:52:14,null,null,MERGE,{predicate -> (sp...,null,null,null,3,Serializable,false,{numTargetRowsCop...,null,Apache-Spark/3.3....
3,2026-02-10 13:08:44,null,null,UPDATE,{predicate -> (id...,null,null,null,2,Serializable,false,{numRemovedFiles ...,null,Apache-Spark/3.3....
2,2026-02-10 13:06:37,null,null,WRITE,"{mode -> Append, ...",null,null,null,1,Serializable,true,"{numFiles -> 2, n...",null,Apache-Spark/3.3....
1,2026-02-10 13:06:17,null,null,WRITE,"{mode -> Append, ...",null,null,null,0,Serializable,true,"{numFiles -> 1, n...",null,Apache-Spark/3.3....
0,2026-02-10 12:56:41,null,null,CREATE OR REPLACE...,{isManaged -> tru...,null,null,null,null,Serializable,true,{},null,Apache-Spark/3.3....


In [15]:
spark.sql('select * from module_4.lesson43 version as of 2')

id,run_params,status,status_message
1,lala1,lala2,lala3
3,lala1,lala2,lala3
2,lala1,lala2,lala3


26/02/11 05:57:13 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_154_16 !
26/02/11 05:57:13 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_116_3 !
26/02/11 05:57:13 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_116_8 !
26/02/11 05:57:13 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_100_26 !
26/02/11 05:57:13 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_100_40 !
26/02/11 05:57:13 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_154_40 !
26/02/11 05:57:13 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_100_44 !
26/02/11 05:57:13 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_116_24 !
26/02/11 05:57:13 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_154_31 !
26/02/11 05:57:13 WARN BlockManagerMasterEndpoint: No more replicas available for rdd_100_1 !
26/02/11 05:57:13 WARN BlockManagerMasterEndpoint: No